# security

In [6]:
import jwt
from datetime import datetime, timedelta
from passlib.context import CryptContext
SECRET_KEY: str = "your-secret-key"

ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def get_password_hash(password: str) -> str:
    return pwd_context.hash(password)

def verify_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)

def create_access_token(data: dict) -> str:
    to_encode = data.copy()
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def decode_access_token(token: str) -> dict:
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload if "exp" in payload else None
    except jwt.PyJWTError:
        return None


In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import json
import base64
def decode_jwt_payload(jwt):
    # Tách phần payload của JWT
    payload = jwt.split(".")[1]
    
    # Chuyển Base64-URL thành Base64 chuẩn
    payload += "=" * (-len(payload) % 4)  # Thêm padding nếu cần
    decoded_bytes = base64.urlsafe_b64decode(payload)  # Giải mã Base64
    
    # Chuyển bytes thành chuỗi JSON
    decoded_str = decoded_bytes.decode("utf-8")
    
    # Parse JSON thành dictionary
    return json.loads(decoded_str)

user = input("Enter your username: ")
passs = input("Enter your password: ")
os.system('cls')

login_data = {
    'username': user,
    'password': passs,
    '_eventId': 'submit',
    'submit': 'Login',
}


In [ ]:
import requests
import base64
import json
import os
from bs4 import BeautifulSoup

def get_information(user, passs):
    login_data = {
        'username': user,
        'password': passs,
        '_eventId': 'submit',
        'submit': 'Login',
    }

    def decode_jwt_payload(jwt):
        # Tách phần payload của JWT
        payload = jwt.split(".")[1]
        
        # Chuyển Base64-URL thành Base64 chuẩn
        payload += "=" * (-len(payload) % 4)  # Thêm padding nếu cần
        decoded_bytes = base64.urlsafe_b64decode(payload)  # Giải mã Base64
        
        # Chuyển bytes thành chuỗi JSON
        decoded_str = decoded_bytes.decode("utf-8")
        
        # Parse JSON thành dictionary
        return json.loads(decoded_str)

    os.system('cls')  # Clear the console

    with requests.Session() as S:
        S.cookies.clear()
        headers = {
            'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36 Edg/126.0.0.0'
        }

        # Login to MyBK
        login_url = 'https://sso.hcmut.edu.vn/cas/login?service=https%3A%2F%2Fmybk.hcmut.edu.vn%2Fapp%2Flogin%2Fcas'
        home_url = 'https://mybk.hcmut.edu.vn/app/'
        r = S.get(login_url, headers=headers)
        soup = BeautifulSoup(r.content, 'html5lib')

        login_data['lt'] = soup.find('input', attrs={'name': 'lt'})['value']
        login_data['execution'] = soup.find('input', attrs={'name': 'execution'})['value']
        r = S.post(login_url, data=login_data)

        if r.url == home_url:
            uni_records_url = 'https://mybk.hcmut.edu.vn/app/he-thong-quan-ly/sinh-vien/ket-qua-hoc-tap'
            response_records = S.get(uni_records_url)
            soup1 = BeautifulSoup(response_records.content, 'html5lib')
            token = soup1.find('input', attrs={'id': 'hid_Token'})['value']

            json_data = decode_jwt_payload(token)

            profiles = json_data['profiles']
            person_ID = json.loads(profiles)['personId']

            url_mark = f"https://mybk.hcmut.edu.vn/api/sinh-vien/xem-ket-qua-hoc-tap/v2?mssv={person_ID}&null"
            url_full_info = f"https://mybk.hcmut.edu.vn/api/v1/student/detail-info-by-code/{person_ID}?null"

            if response_records.status_code == 200:
                headers = {
                    'Authorization': 'Bearer ' + token,
                    'Content-Type': 'application/json'
                }

                response_info = S.get(url_full_info, headers=headers)
                full_info = response_info.json()

                response_info = S.get(url_mark, headers=headers)
                mark = response_info.json()
                data = full_info['data']
                code = data['code']
                lastName = data['lastName']
                firstName = data['firstName']
                orgEmail = data['orgEmail']

                # Kết quả trả về
                info = {
                    'code': code,
                    'lastName': lastName,
                    'firstName': firstName,
                    'orgEmail': orgEmail
}
                return {"status": "success", "full_info": info}
        return {"status": "failed"}


In [30]:
result= get_information("thinh.tangbaobao", "12102004")

In [31]:

result

{'status': 'success',
 'full_info': {'code': '2213307',
  'lastName': 'TĂNG PHỒN',
  'firstName': 'THỊNH',
  'orgEmail': 'thinh.tangbaobao@hcmut.edu.vn'}}